# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from stock_prices_predictions.get_data import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import layers
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mean_absolute_percentage_error
import joblib
from tensorflow.keras.models import load_model

In [2]:
#list_of_10_stocks = ['T', 'INTC', 'ADBE', 'JPM', 'PG', 'NVDA', 'AAPL', 'AMZN', 'UNH', 'MA']
list_of_10_stocks = ['T', 'INTC']

# Loads model and transformer

In [3]:
T_scaler= joblib.load('T_scaler.joblib')
INTC_scaler = joblib.load('INTC_scaler.joblib')
#ADBE_scaler = joblib.load('ADBE_scaler.joblib')
#JPM_scaler = joblib.load('JPM_scaler.joblib')
#PG_scaler = joblib.load('PG_scaler.joblib')
#NVDA_scaler = joblib.load('NVDA_scaler.joblib')
#AAPL_scaler = joblib.load('AAPL_scaler.joblib')
#AMZN_scaler = joblib.load('AMZN_scaler.joblib')
#UNH_scaler = joblib.load('UNH_scaler.joblib')
#MA_scaler = joblib.load('MA_scaler.joblib')

T_model = load_model('T_model.h5')
INTC_model = load_model('INTC_model.h5')
#ADBE_model = load_model('ADBE_model.h5')
#JPM_model = load_model('JPM_model.h5')
#PG_model = load_model('PG_model.h5')
#NVDA_model = load_model('NVDA_model.h5')
#AAPL_model = load_model('AAPL_model.h5')
#AMZN_model = load_model('AMZN_model.h5')
#UNH_model = load_model('UNH_model.h5')
#MA_model = load_model('MA_model.h5')

# Load dataframe

In [4]:
df = get_portfolio_data()

In [5]:
df.dtypes

ticker               object
open                float64
close               float64
adj_close           float64
low                 float64
high                float64
volume                int64
date         datetime64[ns]
sector               object
year                  int64
month                 int64
day                   int64
dtype: object

In [6]:
df.head(5)

,ticker,open,close,adj_close,low,high,volume,date,sector,year,month,day
8588,AAPL,111.389999,109.330002,102.319397,107.349998,111.440002,53204600,2015-01-02,TECHNOLOGY,2015,1,2
8589,AAPL,108.290001,106.250000,99.436890,105.410004,108.650002,64285500,2015-01-05,TECHNOLOGY,2015,1,5
8590,AAPL,106.540001,106.260002,99.446251,104.629997,107.430000,65797100,2015-01-06,TECHNOLOGY,2015,1,6
8591,AAPL,107.199997,107.750000,100.840714,106.699997,108.199997,40105900,2015-01-07,TECHNOLOGY,2015,1,7
8592,AAPL,109.230003,111.889999,104.715256,108.699997,112.150002,59364500,2015-01-08,TECHNOLOGY,2015,1,8


# Create classes

In [7]:
class Stock:
    def __init__(self, name, initial_price, dolars):
        self.name = name
        self.initial_price = initial_price
        self.current_price = initial_price
        self.amount_owned = dolars
        self.prediction = 0.0
        self.prediction_higher = True
        self.prices = 0.0
    
    def myfunc(self):
        print("Hello my name is " + self.name)
    
    def show_price(self):
        print("my initial price is " + str(self.initial_price))
    
    def prediction(self,number):
        self.prediction = number
    
    def prices(self,number):
        self.prices = number
         

# User input

In [8]:
initial_date = '2015-04-15'
final_date = '2015-04-20'
initial_amount = 10000
ticker = "T"

# Instatiate classes

In [9]:
list_stocks = {}
for i in list_of_10_stocks:
    qty = len(list_of_10_stocks)
    price = df[(df["date"]==initial_date) & (df["ticker"]==i)]
    list_stocks[i] = Stock(i, price['adj_close'], initial_amount/qty)

In [10]:
list_stocks

{'T': <__main__.Stock at 0x7f24206f9220>,
 'INTC': <__main__.Stock at 0x7f24206f9370>}

In [11]:
list_stocks['T'].amount_owned

5000.0

# Create predictions

In [12]:
df_T2 = df[df['ticker']== 'T']
df_T2.reset_index(inplace=True)
ind = df_T2.index[df_T2['date'] == initial_date].tolist()
ind2 = df_T2.index[df_T2['date'] == final_date].tolist()
df_T2 = df_T2[ind[0]-18:ind2[0]+1]
df_T2 = df_T2.adj_close

In [13]:
df_T2 = pd.DataFrame(df_T2)
X_test_T = T_scaler.transform(df_T2)
X_test_T2 = create_sequences_scaled(pd.DataFrame(X_test_T))
prediction = T_model.predict(X_test_T2)

prediction_back = T_scaler.inverse_transform(prediction.reshape(-1, 1))

In [14]:
list_stocks['T'].prediction = prediction_back

In [15]:
df_T2 = df[df['ticker']== 'T']
df_T2.reset_index(inplace=True)
ind = df_T2.index[df_T2['date'] == initial_date].tolist()
ind2 = df_T2.index[df_T2['date'] == final_date].tolist()
df_T2 = df_T2[ind[0]:ind2[0]+1]
list_stocks['T'].prices = df_T2

In [16]:
df_T2 = df[df['ticker']== 'INTC']
df_T2.reset_index(inplace=True)
ind = df_T2.index[df_T2['date'] == initial_date].tolist()
ind2 = df_T2.index[df_T2['date'] == final_date].tolist()
df_T2 = df_T2[ind[0]-18:ind2[0]+1]
df_T2 = df_T2.adj_close
df_T2 = pd.DataFrame(df_T2)
X_test_T = INTC_scaler.transform(df_T2)
X_test_T2 = create_sequences_scaled(pd.DataFrame(X_test_T))
prediction = INTC_model.predict(X_test_T2)
prediction_back_INTC = INTC_scaler.inverse_transform(prediction.reshape(-1, 1))

In [17]:
list_stocks['INTC'].prediction = prediction_back_INTC

In [18]:
df_T2 = df[df['ticker']== 'INTC']
df_T2.reset_index(inplace=True)
ind = df_T2.index[df_T2['date'] == initial_date].tolist()
ind2 = df_T2.index[df_T2['date'] == final_date].tolist()
df_T2 = df_T2[ind[0]:ind2[0]+1]

In [19]:
list_stocks['INTC'].prices = df_T2

# Loop through days using models

In [31]:
df_ = df[df['ticker']== 'T']
df_.reset_index(inplace=True)
ind = df_.index[df_['date'] == initial_date].tolist()
ind2 = df_.index[df_['date'] == final_date].tolist()
df_ = df_[ind[0]:ind2[0]+1]

for i in range(len(df_)-1):
    print(list_stocks['T'].prediction[i])
    print(list_stocks['T'].price[i])

[27.351194]


AttributeError: 'Stock' object has no attribute 'price'

In [29]:
list_stocks['T'].prediction

array([[27.351194],
       [27.336378],
       [27.266119]], dtype=float32)

In [32]:
list_stocks['T'].price

AttributeError: 'Stock' object has no attribute 'price'